# होटल बुकिंग में प्राथमिकता सदस्य मिडलवेयर

यह नोटबुक Microsoft Agent Framework का उपयोग करके **फंक्शन-आधारित मिडलवेयर** का प्रदर्शन करती है। हम शर्तीय वर्कफ़्लो उदाहरण को एक मिडलवेयर लेयर जोड़कर बढ़ाते हैं, जो प्राथमिकता सदस्यों को विशेष सुविधाएं प्रदान करती है।

## आप क्या सीखेंगे:
1. **फंक्शन-आधारित मिडलवेयर**: फंक्शन परिणामों को इंटरसेप्ट और संशोधित करना
2. **संदर्भ तक पहुंच**: `context.result` को निष्पादन के बाद पढ़ना और संशोधित करना
3. **व्यावसायिक तर्क कार्यान्वयन**: प्राथमिकता सदस्य लाभ
4. **परिणाम ओवरराइड**: उपयोगकर्ता की स्थिति के आधार पर फंक्शन परिणाम बदलना
5. **एक ही वर्कफ़्लो, अलग-अलग परिणाम**: मिडलवेयर-चालित व्यवहार परिवर्तन

## मिडलवेयर के साथ वर्कफ़्लो आर्किटेक्चर:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## शर्तीय वर्कफ़्लो से मुख्य अंतर:

**मिडलवेयर के बिना** (14-conditional-workflow.ipynb):
- पेरिस में कोई कमरे नहीं → alternative_agent पर रूट करें

**मिडलवेयर के साथ** (इस नोटबुक में):
- नियमित उपयोगकर्ता + पेरिस → कोई कमरे नहीं → alternative_agent पर रूट करें
- प्राथमिकता उपयोगकर्ता + पेरिस → 🌟 मिडलवेयर ओवरराइड करता है! → उपलब्ध → booking_agent पर रूट करें

## आवश्यकताएँ:
- Microsoft Agent Framework इंस्टॉल किया हुआ हो
- शर्तीय वर्कफ़्लो की समझ (देखें 14-conditional-workflow.ipynb)
- GitHub टोकन या OpenAI API कुंजी
- मिडलवेयर पैटर्न की बुनियादी समझ


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## चरण 1: संरचित आउटपुट के लिए Pydantic मॉडल परिभाषित करें

ये मॉडल उस **schema** को परिभाषित करते हैं जिसे एजेंट्स वापस करेंगे। हमने `priority_override` फ़ील्ड जोड़ी है ताकि यह ट्रैक किया जा सके कि कब मिडलवेयर उपलब्धता परिणाम को संशोधित करता है।


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## चरण 2: प्राथमिकता सदस्य डेटाबेस परिभाषित करें

इस डेमो के लिए, हम प्राथमिकता सदस्यता डेटाबेस का अनुकरण करेंगे। उत्पादन में, यह एक वास्तविक डेटाबेस या API से क्वेरी करेगा।

**प्राथमिकता सदस्य:**
- `alice@example.com` - वीआईपी सदस्य
- `bob@example.com` - प्रीमियम सदस्य  
- `priority_user` - परीक्षण खाता


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## चरण 3: होटल बुकिंग टूल बनाएं

पिछले कंडीशनल वर्कफ़्लो की तरह ही, लेकिन अब इसे मिडलवेयर द्वारा इंटरसेप्ट किया जाएगा!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## चरण 4: 🌟 प्राथमिकता जांच मिडलवेयर बनाएं (मुख्य विशेषता!)

यह इस नोटबुक की **मुख्य कार्यक्षमता** है। मिडलवेयर:

1. होटल_booking फ़ंक्शन कॉल को **अवरोधित** करता है
2. `next(context)` को कॉल करके फ़ंक्शन को सामान्य रूप से **निष्पादित** करता है
3. `context.result` में परिणाम की **जांच** करता है
4. यदि उपयोगकर्ता प्राथमिकता वाला है और कमरे उपलब्ध नहीं हैं, तो परिणाम को **ओवरराइड** करता है
5. संशोधित परिणाम को एजेंट को **वापस** करता है

**मुख्य पैटर्न:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## चरण 5: रूटिंग के लिए कंडीशन फंक्शन्स परिभाषित करें

वही कंडीशन फंक्शन्स जो कंडीशनल वर्कफ़्लो में हैं - वे संरचित आउटपुट का निरीक्षण करते हैं ताकि रूटिंग निर्धारित की जा सके।


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## चरण 6: कस्टम डिस्प्ले एग्जीक्यूटर बनाएं

पहले जैसा ही एग्जीक्यूटर - अंतिम वर्कफ़्लो आउटपुट प्रदर्शित करता है।


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## चरण 7: पर्यावरण वेरिएबल्स लोड करें

LLM क्लाइंट (GitHub Models या OpenAI) को कॉन्फ़िगर करें।


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## चरण 8: मिडलवेयर के साथ AI एजेंट बनाएं

**मुख्य अंतर:** जब हम availability_agent बनाते हैं, तो हम `middleware` पैरामीटर पास करते हैं!

यही तरीका है जिससे हम priority_check_middleware को एजेंट के फंक्शन इनवोकेशन पाइपलाइन में डालते हैं।


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## चरण 9: वर्कफ़्लो बनाएं

पहले जैसा ही वर्कफ़्लो ढांचा - उपलब्धता के आधार पर सशर्त रूटिंग।


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## चरण 10: टेस्ट केस 1 - पेरिस में सामान्य उपयोगकर्ता (कोई ओवरराइड नहीं)

एक सामान्य उपयोगकर्ता पेरिस में बुक करने की कोशिश करता है → कोई कमरे उपलब्ध नहीं → वैकल्पिक_एजेंट पर रूट होता है


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## चरण 11: टेस्ट केस 2 - 🌟 प्राथमिकता उपयोगकर्ता पेरिस में (ओवरराइड के साथ!)

एक प्राथमिकता सदस्य पेरिस में बुकिंग करने की कोशिश करता है → शुरुआत में कोई कमरे उपलब्ध नहीं → 🌟 मिडलवेयर ओवरराइड करता है! → बुकिंग एजेंट को रूट करता है

**यह मिडलवेयर की शक्ति का मुख्य प्रदर्शन है!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## चरण 12: टेस्ट केस 3 - स्टॉकहोम में प्राथमिकता उपयोगकर्ता (पहले से उपलब्ध)

प्राथमिकता उपयोगकर्ता स्टॉकहोम की कोशिश करता है → कमरे उपलब्ध हैं → कोई ओवरराइड की आवश्यकता नहीं → बुकिंग एजेंट को रूट करता है

यह दिखाता है कि मिडलवेयर केवल तब कार्य करता है जब आवश्यकता होती है!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## मुख्य बातें और मिडलवेयर की अवधारणाएँ

### ✅ आपने क्या सीखा:

#### **1. फ़ंक्शन-आधारित मिडलवेयर पैटर्न**

मिडलवेयर एक साधारण async फ़ंक्शन का उपयोग करके फ़ंक्शन कॉल को इंटरसेप्ट करता है:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. संदर्भ तक पहुंच और परिणाम को ओवरराइड करना**

- `context.function` - कॉल किए जा रहे फ़ंक्शन तक पहुंच
- `context.arguments` - फ़ंक्शन के तर्क पढ़ें
- `context.kwargs` - अतिरिक्त पैरामीटर तक पहुंच
- `await next(context)` - फ़ंक्शन को निष्पादित करें
- `context.result` - फ़ंक्शन के आउटपुट को पढ़ें/संशोधित करें

#### **3. व्यावसायिक तर्क का कार्यान्वयन**

हमारा मिडलवेयर प्राथमिकता वाले सदस्यों के लाभ लागू करता है:
- **सामान्य उपयोगकर्ता**: कोई संशोधन नहीं, मानक वर्कफ़्लो
- **प्राथमिकता उपयोगकर्ता**: "कोई उपलब्धता नहीं" → "उपलब्ध" को ओवरराइड करें
- **शर्तीय तर्क**: केवल आवश्यकता होने पर ओवरराइड करता है

#### **4. वही वर्कफ़्लो, अलग-अलग परिणाम**

मिडलवेयर की शक्ति:
- ✅ वर्कफ़्लो संरचना में कोई बदलाव नहीं
- ✅ टूल फ़ंक्शन में कोई बदलाव नहीं
- ✅ शर्तीय रूटिंग तर्क में कोई बदलाव नहीं
- ✅ केवल मिडलवेयर → अलग व्यवहार!

### 🚀 वास्तविक दुनिया में उपयोग:

1. **वीआईपी/प्रीमियम सुविधाएँ**
   - प्रीमियम उपयोगकर्ताओं के लिए दर सीमा को ओवरराइड करें
   - संसाधनों तक प्राथमिकता पहुंच प्रदान करें
   - प्रीमियम सुविधाओं को गतिशील रूप से अनलॉक करें

2. **ए/बी परीक्षण**
   - उपयोगकर्ताओं को अलग-अलग कार्यान्वयन पर रूट करें
   - विशिष्ट उपयोगकर्ताओं के साथ नई सुविधाओं का परीक्षण करें
   - सुविधाओं को धीरे-धीरे रोल आउट करें

3. **सुरक्षा और अनुपालन**
   - फ़ंक्शन कॉल का ऑडिट करें
   - संवेदनशील ऑपरेशनों को ब्लॉक करें
   - व्यावसायिक नियम लागू करें

4. **प्रदर्शन अनुकूलन**
   - विशिष्ट उपयोगकर्ताओं के लिए परिणाम कैश करें
   - संभव होने पर महंगे ऑपरेशनों को छोड़ें
   - गतिशील संसाधन आवंटन

5. **त्रुटि प्रबंधन और पुनः प्रयास**
   - त्रुटियों को सहजता से पकड़ें और संभालें
   - पुनः प्रयास तर्क लागू करें
   - वैकल्पिक कार्यान्वयन पर वापस जाएं

6. **लॉगिंग और निगरानी**
   - फ़ंक्शन निष्पादन समय को ट्रैक करें
   - पैरामीटर और परिणाम लॉग करें
   - उपयोग पैटर्न की निगरानी करें

### 🔑 डेकोरेटर से मुख्य अंतर:

| विशेषता | डेकोरेटर | मिडलवेयर |
|---------|-----------|------------|
| **स्कोप** | एकल फ़ंक्शन | एजेंट के सभी फ़ंक्शन |
| **लचीलापन** | परिभाषा पर तय | रनटाइम पर गतिशील |
| **संदर्भ** | सीमित | पूर्ण एजेंट संदर्भ |
| **संयोजन** | कई डेकोरेटर | मिडलवेयर पाइपलाइन |
| **एजेंट-अवेयर** | नहीं | हाँ (एजेंट की स्थिति तक पहुंच) |

### 📚 मिडलवेयर का उपयोग कब करें:

✅ **मिडलवेयर का उपयोग करें जब:**
- आपको उपयोगकर्ता/सत्र स्थिति के आधार पर व्यवहार संशोधित करना हो
- आप कई फ़ंक्शनों पर तर्क लागू करना चाहते हैं
- आपको एजेंट-स्तरीय संदर्भ तक पहुंच चाहिए
- आप क्रॉस-कटिंग चिंताओं (लॉगिंग, ऑथ, आदि) को लागू कर रहे हैं

❌ **मिडलवेयर का उपयोग न करें जब:**
- साधारण इनपुट मान्यता (Pydantic का उपयोग करें)
- फ़ंक्शन-विशिष्ट तर्क (फ़ंक्शन में रखें)
- एक बार के संशोधन (बस फ़ंक्शन बदलें)

### 🎓 उन्नत पैटर्न:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 संबंधित अवधारणाएँ:

- **एजेंट मिडलवेयर**: agent.run() कॉल को इंटरसेप्ट करता है
- **फ़ंक्शन मिडलवेयर**: टूल फ़ंक्शन कॉल को इंटरसेप्ट करता है (जो हमने उपयोग किया!)
- **मिडलवेयर पाइपलाइन**: क्रम में निष्पादित होने वाले मिडलवेयर की श्रृंखला
- **संदर्भ प्रचार**: मिडलवेयर श्रृंखला के माध्यम से स्थिति पास करें



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
